In [44]:
from bs4 import BeautifulSoup
import requests
import json
from requests import Session
import re
from concurrent.futures import ThreadPoolExecutor
import pandas as pd

def get_house_info(url,session):
  house_info = session.get(url)
  soup = BeautifulSoup(house_info.text, "html.parser")


  house_info={}
  # captures {<any text>} within window.classified
  regex = r"window.classified = (\{.*\})"
  

  # returns a list containing {<any text}
  script_text=soup.find('div',attrs={"id":"main-container"}).script.text
  # applies regex to scrip_text
  script_after_regex = re.findall(regex, script_text)
  
  # returns first element of script_after_regex (full clustered dictionary of property attributes)
  script_text_dict = json.loads(script_after_regex[0])
  properties = script_text_dict['property']
  
  # adds 'key : value' pairs to house_info dictionary. If value does not exist, it is replaced by a 0.
  try:
    house_info['id'] = script_text_dict['id']
  except:
    house_info['id'] = 0

  try:
    house_info['type of property'] = properties['type']
  except:
    house_info['type of property'] = 0

  try:
    house_info['subtype of property'] = properties['subtype']
  except:
    house_info['subtype of property'] = 0

  try:
    house_info['locality'] = properties['location']['locality']
  except:
    house_info['locality'] = 0

  try:
    house_info['price'] = script_text_dict['price']['mainValue']
  except:
    house_info['price'] = 0
  
  try:
    house_info['type of sale'] = script_text_dict['price']['type']
  except:
    house_info['type of sale'] = 0

  try:  
    house_info['number of bedrooms'] = properties['bedroomCount']
  except:
    house_info['number of bedrooms'] = 0

  try:  
    house_info['living area'] = properties['livingRoom']['surface']
  except:
    house_info['living area'] = 0

  try:
    house_info['fully equipped kitchen'] = script_text_dict['property']['kitchen']['type'].lower()
  except:
    house_info['fully equipped kitchen'] = 0

  try:
    house_info['furnished'] = script_text_dict['transaction']['sale']['isFurnished']
  except:
    house_info['furnished'] = 0

  try:
    house_info['open fire'] = properties['fireplaceExists']
  except:
    house_info['open fire'] = 0

  try:
    house_info['terrace'] = properties['hasTerrace']
  except:
    house_info['terrace'] = 0

  try:
    house_info['terrace area'] = properties['terraceSurface']
  except:
    house_info['terrace area'] = 0
  
  try:
    house_info['garden'] = properties['hasGarden']
  except:
    house_info['garden'] = 0

  try:
    house_info['garden area'] = properties['gardenSurface']
  except:
    house_info['garden area'] = 0

  try:
    house_info['total property area'] = properties['netHabitableSurface']
  except:
    house_info['total property area'] = 0

  try:
    house_info['total land area'] = properties['land']['surface']
  except:
    house_info['total land area'] = 0

  try:
    house_info['number of facades'] = properties['building']['facadeCount']
  except:
    house_info['number of facades'] = 0

  try:
    house_info['swimming pool'] = properties['hasSwimmingPool']
  except:
    house_info['swimming pool'] = 0

  try:
    house_info['state of the building'] = properties['building']['condition'].lower()
  except: 
    house_info['state of the building'] = 0



  if house_info['fully equipped kitchen'] == "undefined" or house_info['fully equipped kitchen'] == "uninstalled" or house_info['fully equipped kitchen'] == "semi_equipped" or house_info['fully equipped kitchen'] == "usa_uninstalled" or house_info['fully equipped kitchen'] == "usa_semi_equipped":
    house_info['fully equipped kitchen'] = 0
  else:
    house_info['fully equipped kitchen'] = 1
  for key in house_info.keys():
    if house_info[key] == None:
      house_info[key] = 0
    elif house_info[key] == False:
      house_info[key] = 0
    elif house_info[key] == True:
      house_info[key] = 1
    else:
      pass
  return house_info




In [2]:
"""key_list = ['type', 'subtype' , 'locality' , 'price' , 'sale type' , 'number of bedrooms' , 'living room area' , 'kitchen status' , 'furnished' , 'fireplace' , 'terrace' , 
            'terrace area' , 'garden' , 'garden area' , 'total land area' , 'facades' , 'swimming pool' , 'current state']
    value_list = [script_text_dict['id'] , properties['type'] , properties['subtype'] , properties['location']['locality'] , script_text_dict['price']['mainValue'] ,
                  script_text_dict['price']['type'] , properties['bedroomCount'] ,  properties['livingRoom']['surface'] , properties['livingRoom']['surface'] , 
                  script_text_dict['property']['kitchen']['type'] , script_text_dict['transaction']['sale']['isFurnished'] , properties['fireplaceExists'] , 
                  properties['fireplaceExists'] , properties['hasTerrace'] , properties['terraceSurface'] , properties['hasGarden'] , properties['gardenSurface'] ,
                  properties['netHabitableSurface'] , properties['land']['surface'] , properties['building']['facadeCount'] , properties['hasSwimmingPool'] , 
                  properties['building']['condition']]
    print(key_list)
    print(value_list)
    for i in range(len(key_list)):
        try:
          house_info[key_list[i]] = value_list[i]
        except:
            house_info[key_list[i]] = 0"""

"key_list = ['type', 'subtype' , 'locality' , 'price' , 'sale type' , 'number of bedrooms' , 'living room area' , 'kitchen status' , 'furnished' , 'fireplace' , 'terrace' , \n            'terrace area' , 'garden' , 'garden area' , 'total land area' , 'facades' , 'swimming pool' , 'current state']\n    value_list = [script_text_dict['id'] , properties['type'] , properties['subtype'] , properties['location']['locality'] , script_text_dict['price']['mainValue'] ,\n                  script_text_dict['price']['type'] , properties['bedroomCount'] ,  properties['livingRoom']['surface'] , properties['livingRoom']['surface'] , \n                  script_text_dict['property']['kitchen']['type'] , script_text_dict['transaction']['sale']['isFurnished'] , properties['fireplaceExists'] , \n                  properties['fireplaceExists'] , properties['hasTerrace'] , properties['terraceSurface'] , properties['hasGarden'] , properties['gardenSurface'] ,\n                  properties['netHabitableSurfac

#### Attributes to include in a single property's data.
* Locality
* Type of property (House/apartment)
* Subtype of property (Bungalow, Chalet, Mansion, ...)
* Price
* Type of sale (Exclusion of life sales)
* Number of bedrooms
* Living Area
* Fully equipped kitchen (Yes/No)
* Furnished (Yes/No)
* Open fire (Yes/No)
* Terrace (Yes/No)
* If yes: Area
* Garden (Yes/No)
* If yes: Area
* Surface of the land
* Surface area of the plot of land 
* Number of facades
* Swimming pool (Yes/No)
* State of the building (New, to be renovated, ...)

In [42]:
def get_url(session, i=1):
    root_house_list_url = f"https://www.immoweb.be/en/search/house/for-sale?countries=BE&isNewlyBuilt=false&isALifeAnnuitySale=false&isAPublicSale=false&page={i}&orderBy=relevance"
    root_apartment_list_url = f"https://www.immoweb.be/en/search/apartment/for-sale?countries=BE&isNewlyBuilt=false&isAPublicSale=false&isALifeAnnuitySale=false&isUnderOption=false&isAnInvestmentProperty=false&page={i}&orderBy=relevance"

    house_list = session.get(root_house_list_url)
    apartment_list = session.get(root_apartment_list_url)

    soup_house = BeautifulSoup(house_list.text, "html.parser")
    soup_apartment = BeautifulSoup(apartment_list.text, "html.parser")
    url_list = []

    for block in soup_house.find_all("h2",{'class':"card__title card--result__title"}):
        tag_a = block.find('a')['href']
        url_list.append(tag_a)
    
    for block in soup_apartment.find_all("h2",{'class':"card__title card--result__title"}):
        tag_a = block.find('a')['href']
        url_list.append(tag_a)
    
    return(url_list)



In [43]:
def get_all_urls(page = 200):
    """this function takes one parameter 'page'(int), returns a list of urls scraped from\
        immoweb search page. The list contains page*60 urls in total."""
    with ThreadPoolExecutor(max_workers=12) as executor:
        with requests.Session() as session:
            futures = [executor.submit(get_url, session, i) for i in range(1,page+1)]
            list_of_url_temp= [item.result() for item in futures]
            final_url_list = [element for innerList in list_of_url_temp for element in innerList]
    return final_url_list


In [50]:
def scraper(final_url_list): 
    """This function takes one parameter final_url_list, returns a list of dictionary scraped from\
        immoweb page."""       
    data_list = []
    futures = []
    #build up pool of tasks to be executed
    with ThreadPoolExecutor(max_workers=15) as executor:
        with requests.Session() as session:
            for url in final_url_list:
                futures.append(executor.submit(get_house_info,url,session))
            #try to excute the get_house_info function, append the result to final list
            #capture the errors and print out the url
            for item in futures:
                try:
                    data_list.append(item.result())
                except:
                    i = futures.index(item)
                    print(f"There is an error while scraping this website:{final_url_list[i]}")
    return data_list

error at list index out of range
error at list index out of range
error at list index out of range
error at list index out of range
error at list index out of range
error at list index out of range
error at list index out of range
error at list index out of range
error at list index out of range
error at list index out of range
error at list index out of range
error at list index out of range
error at list index out of range
error at list index out of range


In [48]:

df = pd.DataFrame(data_list)
df.to_csv('output.csv', index=False)
df.info()


,id,price,number of bedrooms,living area,fully equipped kitchen,furnished,open fire,terrace,terrace area,garden,garden area,total property area,total land area,number of facades,swimming pool
count,1.199400e+04,1.199400e+04,11994.000000,11994.000000,11994.000000,11994.000000,11994.000000,11994.000000,11994.000000,11994.000000,11994.000000,11994.000000,11994.000000,11994.000000,11994.000000
mean,1.052557e+07,5.227648e+05,2.990662,19.723362,0.872353,0.049608,0.337752,0.706603,12.643488,0.368434,188.888611,170.656161,680.440804,2.141321,0.033433
std,2.272668e+05,5.755820e+05,1.733445,39.755246,0.333710,0.217143,0.472963,0.455338,28.692793,0.482400,1614.444430,134.963545,6220.654272,1.403069,0.179773
min,7.710166e+06,2.000000e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.050019e+07,2.400000e+05,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,90.000000,0.000000,2.000000,0.000000
50%,1.059708e+07,3.500000e+05,3.000000,16.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,135.000000,0.000000,2.000000,0.000000
75%,1.064203e+07,5.750000e+05,4.000000,34.000000,1.000000,0.000000,1.000000,1.000000,15.000000,1.000000,0.000000,210.000000,450.000000,3.000000,0.000000
max,1.066975e+07,1.100000e+07,46.000000,3558.000000,1.000000,1.000000,1.000000,1.000000,800.000000,1.000000,87000.000000,1500.000000,500000.000000,25.000000,1.000000


In [49]:
#remove empty IDs
df.sort_values('id', ascending= False)
df

,id,type of property,subtype of property,locality,price,type of sale,number of bedrooms,living area,fully equipped kitchen,furnished,open fire,terrace,terrace area,garden,garden area,total property area,total land area,number of facades,swimming pool,state of the building
0,10668977,HOUSE,HOUSE,Seraing,149000,residential_sale,2,14,0,0,0,1,0,1,25,105,105,2,0,good
1,10669506,HOUSE,HOUSE,Anderlecht,399000,residential_sale,4,0,0,0,0,1,10,1,140,170,210,2,0,to_be_done_up
2,10669505,HOUSE,VILLA,Wavre,435000,residential_sale,4,57,1,0,0,1,30,1,526,185,748,4,0,good
3,10669507,HOUSE,EXCEPTIONAL_PROPERTY,Anderlecht,960000,residential_sale,6,70,1,0,0,1,40,1,290,400,1090,3,0,as_new
4,10666853,HOUSE,HOUSE,Saint-Nicolas,319000,residential_sale,3,29,1,0,0,1,34,1,82,194,197,2,0,as_new
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11989,10611238,APARTMENT,APARTMENT,Oostende,225000,residential_sale,2,0,1,0,0,0,0,0,0,80,0,0,0,0
11990,10318446,APARTMENT,DUPLEX,Oostende,225000,residential_sale,2,45,1,0,0,1,12,0,0,0,0,0,0,0
11991,10630428,APARTMENT,APARTMENT,Anderlecht,239000,residential_sale,2,28,1,0,0,1,40,0,0,90,0,2,0,good
11992,10444277,APARTMENT,APARTMENT,Beveren-Leie,242900,residential_sale,2,0,1,0,0,1,16,0,0,65,0,0,0,0
